<font size="18"><font color = "grey"> Magnetic Data Processing and Visualizations
    </font>

# Notebook Purpose:

To process and visualize raw magnetic survey data collected using a Geometrics G-864 Cs-vapor magnetometer. Code alteration of the load data cell (processing section) will be required if a different magnetometer is used. The notebook includes codes to reduce the terrestrial magnetic data collected over a target and visualize the output of each processing step. The reductions performed in this notebook show one way raw magnetic data can be processed to identify anomalies associated with buried magnetized bodies.
<br>
<br>
<font color = "blue">
<b><i>Reductions performed are:</i></b>
    </font>

1) <b>Detrending of Earth's Magnetic Field </b>
    
2) <b>Removal Of Data Spikes </b>
    
3) <b>Normalization of Data </b>

4) <b>Rolling Average Filtering and Interpolation </b>
    
5) <b>Coordinate Transformation </b>

<br>

<font size="5"><b><u>Input Data Format</u><b></font>

<b><font size="4">File type required:</b>
    <br>
    <br>
    
<font color = "red">
<b><font size="4">csv</b> (comma-seperated values, no UTF-8 encoding)
</font>
<br>  
    <br>
    
<b><font size="4">Required Column Names:</b>
</font>
                  
<b>Counter:</b> numeric (int64)

<b>Date:</b> numeric (object)

<b>Time:</b> MM/DD/YYYY   (object)

<b>LatitudeSensor1:</b> HH:MM (float64)

<b>LongitudeSensor1:</b> Minutes (float64)

<b>MagSensor1:</b> decimal degrees (float64)

<b>Signal1:</b> decimal (int64)


### Import Python Libraries/Modules

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import requests
from pyproj import Proj
import scipy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles
from scipy.interpolate import griddata
import matplotlib.colors as colors
import math as ma
import seaborn as sb
from scipy import interpolate
from matplotlib.patches import Rectangle
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import matplotlib.image as image

<br>
<br>
<font size="18"><font color = "blue"> Processing Functions </font>

### Import data

In [17]:
def import_data(data_file):
    #Read data_in file 
    file = pd.read_csv(data_file)

    return file

### Reproject lat/lon to UTM

Converts Lat/Lon coordinates to UTM coordinates for plotting purposes.

In [18]:
def reproject(data, myProj, inverse):
    #List to store longitudes/latitudes values from reprojection function
    lon_list = []
    lat_list = []

    #List to store eastings/northings values from reprojection function
    east_list = []
    north_list = []
    
    #if = use for UTM easting/northing to lon/lat
    if inverse == 'True':
        for i in range(len(data.easting)):
            lons,lats = myProj(data.easting[i],data.northing[i], inverse=True)
            lon_list.append(int(lons))
            lat_list.append(int(lats))
            
        #Creates new columns for lon/lat and mag values       
        data['lon'] = lon_list
        data['lat'] = lat_list
  
    #elif = use for lon/lat to UTM easting/northing
    elif inverse == 'False':
        for i in range(len(data.lon)):
            eastings,northings = myProj(data.lon[i],data.lat[i], inverse=False)
            east_list.append(int(eastings))
            north_list.append(int(northings))

        #Creates new columns for lon/lat and mag values    
        data['easting'] = east_list
        data['northing'] = north_list
        
    print('REPROJECTION IS COMPLETED')
    return reproject

### Detrend total magnetic field data
Acquires the reference field from NOAA Magnetic Field Calculator via an API key using an averaged latitude and longitude and the date the survey was performed. This mean magnetic value is then subtracted from each data point.

In [19]:
def mag_detrend(data, value=None, date=None, elevation=None):
    startMonth,startDay,startYear = data.date.max().split("/")
    #print('Start',startMonth,startDay,startYear)
    
    mag = data.total_field
    endMonth,endDay,endYear = data.date.min().split("/")  
    mean_lat = np.mean(data.lat.values)
    mean_lon = np.mean(data.lon.values)

    detended_mag = [] 
    
    params = {
        "lat":str(np.mean(data.lat.values)),
        "lon": str(np.mean(data.lon.values)),
        "elev": str(np.mean(data.el.values)),
        "year_0": str(startYear),
        "month_0": str(startMonth),
        "day_0": str(startDay),
        "year_1": str(endYear),
        "month_1": str(endMonth),
        "day_1": str(endDay)
        }

    url = 'https://www.ngdc.noaa.gov/geomag-web/calculators/calculateIgrfwmm?lat1=%s&lon1=%s&elevation=%s&startYear=%s&startMonth=%s&startDay=%s&endYear=%s&endMonth=%s&endDay=%s&elevationUnits=M&coordinateSystem=D&model=IGRF&&key=EAU2y&resultFormat=json'%(
        params["lat"], params["lon"], params["elev"], params["year_0"],params["month_0"],params["day_0"],params["year_1"],params["month_1"],params["day_1"])
    
    global_mag = requests.get(url)
    
    if global_mag.status_code == 200:
        value = global_mag.json()['result'][0]['totalintensity']
        data['detrended_mag'] = mag
        data.detrended_mag -= value
        
    else:
        print('Status code:'+str(global_mag.status_code)+', code should be 200')
        exit()  
        
    print('DETRENDING IS COMPLETED')
    return mag_detrend

### Despike filter (replaces values higher or lower than a cutoff with NaN
Very large magnitude spikes can occur due to the orientaiton of the magnetometer relative to the direction of the Earth's magnetic field. This code removes the spikes by applying user defined min/max cutoff values and replaces the spike values with NaN (not a number).

In [20]:
def despike_data(data, mag_min, mag_max):

    min_value = mag_min
    max_value = mag_max
    
    despiked_data = data.copy()
    despiked_data['cleaned_mag'] = despiked_data['detrended_mag'] 
    despiked_data.loc[despiked_data['cleaned_mag'] < mag_min, 'cleaned_mag'] = np.nan
    despiked_data.loc[despiked_data['cleaned_mag'] > mag_max, 'cleaned_mag'] = np.nan
    
    print('DESPIKING COMPLETED')
    return despiked_data

### Normalize reduced magnetic data
To reduce the magnetic anaomly to near zero, the mean of the dataset is removed from all points in the survey.

In [21]:
#MAKE A DEFINITION FOR NORMALIZING DATA
def normalize_mag(mag):
    norm_mag = []
    mag_mean = np.mean(mag)
    for i in range(len(mag)):
        normalized_mag_point = mag[i] - mag_mean
        norm_mag.append(normalized_mag_point)
        
    print('NORMALIZATION COMPLETED')    
    return norm_mag

### Rolling Average filter
Magnetic data is can be quite "noisy", especially in volcanic terrains. These areas often host an abundance of surface flows, whose remnant magnetization varies with composition. The high frequency oscillations created by these features need removed/smoothed from the anomaly data for interpretation.

In [22]:
def rolling_average(data, direction, seconds, points):
    rolling_average_data = data.copy()
    rolling_average_data.index = pd.to_datetime(100*rolling_average_data.index,unit='ms')
    rolling_average_data = rolling_average_data.resample(str(seconds)).median()
    rolling_average_data = rolling_average_data.rolling(points,center=False).mean()
    rolling_average_data.reset_index(inplace=True)
    
    print('ROLLING AVERAGE COMPLETED')
    return rolling_average_data

<br>
<br>
<font size="18"><font color = "blue"> Visualization Functions </font>

### Field area map with transects

#### For UTM 

In [23]:
def scale_bar(ax, length, location=(0.5, 0.05), linewidth=3):
    '''
    ax is the axes to draw the scalebar on.
    length is the length of the scalebar in km.
    location is center of the scalebar in axis coordinates.
    (ie. 0.5 is the middle of the plot)
    linewidth is the thickness of the scalebar.
    '''
    crs = ccrs.UTM(zone="12N")
    #Get the limits of the axis in lat long
    llx0, llx1, lly0, lly1 = ax.get_extent(crs)
    #Make tmc horizontally centred on the middle of the map,
    #vertically at scale bar location
    #sbllx = (llx1 + llx0) / 2
    #sblly = lly0 + (lly1 - lly0) * location[1]
    #tmc = ccrs.UTM(zone="12N", area_of_use = (sbllx, sblly))
    #Get the extent of the plotted area in coordinates in metres
    x0, x1, y0, y1 = ax.get_extent(crs)
    #Turn the specified scalebar location into coordinates in metres
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    #Calculate a scale bar length if none has been given
    #(Theres probably a more pythonic way of rounding the number but this works)
    if not length: 
        length = (x1 - x0) / 5000 #in km
        ndim = int(np.floor(np.log10(length))) #number of digits in number
        length = round(length, -ndim) #round to 1sf
        #Returns numbers starting with the list
        def scale_number(x):
            if str(x)[0] in ['1', '2', '5']: return int(x)        
            else: return scale_number(x - 10 ** ndim)
        length = scale_number(length) 

    #Generate the x coordinate for the ends of the scalebar
    bar_xs = [sbx - length * 500, sbx + length * 500]
    #Plot the scalebar
    ax.plot(bar_xs, [sby, sby], transform=crs, color='white', linewidth=linewidth)
    #Plot the scalebar label
    ax.text(sbx, sby, str(length) + ' km', transform=crs,
            horizontalalignment='center', color='white',verticalalignment='bottom', fontsize = 16)

def label_utm_grid():
    ''' Warning: should only use with small area UTM maps '''
    x_coords = []
    y_coords = []
    
    ax = plt.gca()    
    for val,label in zip(ax.get_xticks(), ax.get_xticklabels()):
        label.set_text(str(val))
        label.set_position((val,0))  
        x_coords.append(val)
    
    for val,label in zip(ax.get_yticks(), ax.get_yticklabels()):   
        label.set_text(str(val))
        label.set_position((0,val)) 
        y_coords.append(val)
    
    plt.tick_params(bottom=True,top=True,left=True,right=True,
            labelbottom=True,labeltop=False,labelleft=True,labelright=False)
    
    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    plt.grid(True)
    
    return x_coords, y_coords
    
    
def plot_transects_UTM(data, data_2, zoom, buffer, style):
    fig = plt.figure(figsize=(20,15))
    tiler = GoogleTiles(style=style)
    mercator = tiler.crs
    crs = ccrs.UTM(zone="12N")
    ax = plt.axes(projection=crs)
   
    
    #Set map extent
    easting = data.easting
    northing = data.northing
    easting_min = min(easting) - buffer
    easting_max = max(easting) + buffer
    northing_min = min(northing) - buffer
    northing_max = max(northing) + buffer
    extent = (easting_min, easting_max, northing_min, northing_max)
    ax.set_extent(extent,crs=crs)

    ax.add_image(tiler, zoom)
        
    plt.plot(easting, northing, marker='.', color='k', linestyle='None',
             markersize=0.2, alpha=0.7, transform=crs)
    
    plt.plot(data_2.easting, data_2.northing, marker='.', color='whitesmoke', 
             linestyle='-', markersize=0.1, alpha=0.7, transform=crs)
    
         
    x_lines, y_lines = label_utm_grid()
    
    for i in range(len(y_lines)):
        plt.hlines(y_lines[i], easting_min, easting_max, linewidth=0.5, color='k')
    for i in range(len(x_lines)):
        plt.vlines(x_lines[i], northing_min, northing_max, linewidth=0.5, color='k')
    
    ax.ticklabel_format(useOffset=False, style='plain')
        
    #plot a N array
    x, y, arrow_length = 0.1, 0.15, .03
    ax.annotate('N', xy=(x, y), xytext=(x, y-(arrow_length*2)),family='sans-serif', 
                color ='white', 
                arrowprops=dict(facecolor='white',width=5, headwidth=15), 
                ha='center', va='center', 
                fontsize=28, xycoords=ax.transAxes)
    
    plt.xlabel('Easting (m)' )
    plt.ylabel('Northing (m)')

    scale_bar(ax, 2)
    
    plt.show()
    
    return fig

#### Plot profiles

In [24]:
def plot_profile(x, y, direction, name):
    fig = plt.figure(figsize=(15, 7))
    
    if direction == 'ns':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Northing (m)')
        plt.axhline(0,color='r',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        plt.plot(x,y, linestyle='-', color='blue')
       
    elif direction == 'ew':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Easting (m)')
        plt.axhline(0,color='r',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        plt.plot(x,y, linestyle='-', color='blue')
    
    return fig

#### Rolling average profiles

In [25]:
def rolling_profile(data, direction, name):
    fig = plt.figure(figsize=(15, 7))

    if direction == 'ns':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Northing (m)')
        plt.axhline(0,color='r',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        plt.plot(data.northing,data.cleaned_mag, linestyle='-', color='blue')
    
    elif direction == 'ew':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Easting (m)')
        plt.axhline(0,color='r',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        plt.plot(data.easting,data.cleaned_mag,linestyle='-', color='blue')

    return fig

#### Rolling average and topo profile with map inset

In [26]:
def complete_profile(data, map_image, zoom, position, box, direction, name):
    fig = plt.figure(figsize=(15, 7))
    ax = fig.add_subplot()

    if direction == 'ns':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Northing (m)')
        plt.axhline(0,color='k',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        imagebox = OffsetImage(map_image, zoom = zoom)
        #Annotation box for solar pv logo
        #Container for the imagebox referring to a specific position *xy*.
        ab = AnnotationBbox(imagebox, (position[0], position[1]), frameon = False)
        ax.add_artist(ab)
        #add rectangle to plot
        ax.add_patch(Rectangle((box[0], box[1]), box[2], box[3], ls="--", ec ="red", fill=False))
        plt.plot(data.northing,data.cleaned_mag, linestyle='-', color='blue')
    
    elif direction == 'ew':
        plt.title(str(name))
        plt.ylabel('Magnetic Anomaly (nT)')
        plt.xlabel('Easting (m)')
        plt.axhline(0,color='k',linestyle='--')
        plt.ticklabel_format(useOffset=False, style='plain')
        imagebox = OffsetImage(map_image, zoom = zoom)
        #Annotation box for solar pv logo
        #Container for the imagebox referring to a specific position *xy*.
        ab = AnnotationBbox(imagebox, (position[0], position[1]), frameon = False)
        ax.add_artist(ab)
        #add rectangle to plot
        ax.add_patch(Rectangle((box[0], box[1]), box[2], box[3], ls="--", ec ="red", fill=False))
        plt.plot(data.easting,data.cleaned_mag,linestyle='-', color='blue')

    return fig

<br>

<font size="12"><font color = "red"> Processing Inputs and Visualizations
    </font>

## Load data

In [27]:
#Set input file
in_file = './input_data/TB_mag_transect_E-W.csv'

#Open input data file 
data_in = pd.read_csv(in_file, sep=',', low_memory=False )
data_in = data_in[['Counter', 'Date', 'Time', 'LatitudeSensor1',
                   'LongitudeSensor1', 'MagSensor1', 'Signal1']]

data_in.rename(columns={'Counter': 'id', 'Date': 'date', 'Time': 'time',
                        'LatitudeSensor1': 'lat','LongitudeSensor1': 'lon',
                        'MagSensor1': 'total_field', 'Signal1': 'el' }, inplace=True)

print(data_in.head(1))

      id      date     time        lat         lon  total_field    el
0  15152  7/6/2022  26:11.0  43.988774 -112.296544    53093.426  1458


## Reproject coordinates

In [28]:
#REPROJECTION INPUTS
#** always check the proper UTM zone (default = North,12 (Midwest U.S))** - Only CHANGE REPROJECTION INPUTS IF NEEDED
zone = 12
projection = 'utm'
ellipse = 'WGS84'
units = 'm'
definitions = 'no_defs'
hemisphere = 'north'

#TRASNFORMATION DIRECTION
#inverse='True' #for UTM to Lat/Lon
inverse='False' #for Lat/Lon to UTM

#REPROJECTION FUNCTION
myProj = Proj("+proj="+str(projection)+' '+"+zone="+str(zone)+\
              ', '+'+'+str(hemisphere)+' '+"+ellps="+str(ellipse)+\
              ' '+"+datum="+str(ellipse)+' '+"+units="+str(units))

reproject(data_in, myProj, inverse)
print(data_in.head(1))

CRSError: Invalid projection: +proj=utm +zone=12, +north +ellps=WGS84 +datum=WGS84 +units=m +type=crs: (Internal Proj Error: proj_create: Error 1027 (Invalid value for an argument): utm: Invalid value for zone)

#### Profile of total field magnetic anomaly data

In [29]:
#Set transect direciton
direction = 'ew' #or 'ns'

if direction == 'ns':
    x = data_in.northing
elif direction == 'ew':
    x = data_in.easting
    
y = data_in.total_field
name = 'Raw Magnetic Anomlay vs. Distance'
raw_profile = plot_profile(x, y, direction, name)

AttributeError: 'DataFrame' object has no attribute 'easting'

#### Save raw data profile 

In [ ]:
raw_profile.savefig('./profiles/raw_profile.png')

## Detrend total field magnetic anomaly

In [ ]:
mag_detrend(data_in)
print(data_in.head(1))

#### Profile of detrended magnetic anaomly data

In [ ]:
y = data_in.detrended_mag
name = 'Detrended Magnetic Anomlay vs. Distance'
detrended_profile = plot_profile(x, y, direction, name)

#### Save detrended mag data profile

In [ ]:
detrended_profile.savefig('./profiles/detrended_profile.png')

#### Save detrended data

In [ ]:
data_in.to_csv('./output_data/detrended_data.csv') 

## Despike data with min and max boundaries

#### Min and Max magnetic anomaly values

In [ ]:
print('Minimun magnetic anomlay value:',
      min(data_in.detrended_mag))
print('Second lowest magnetic anomaly:',
      data_in.detrended_mag.nsmallest(2).iloc[-1])
print( )
print('Maximum magnetic anomlay value:',
      max(data_in.detrended_mag))
print('Second highest magnetic anomaly:',
      data_in.detrended_mag.nlargest(2).iloc[-1])   

In [ ]:
magnetic_amplitude_min_value = -4000
magnetic_amplitude_max_value = 4000

data_despiked = despike_data(data_in, magnetic_amplitude_min_value, magnetic_amplitude_max_value)

#### Plot despiked data

In [ ]:
if direction == 'ns':
    x = data_despiked.northing
elif direction == 'ew':
    x = data_despiked.easting

y = data_despiked.cleaned_mag
name = 'Despiked Magnetic Anomlay vs. Distance'
despiked_profile = plot_profile(x, y, direction, name)

#### Save despiked profile

In [ ]:
despiked_profile.savefig('./profiles/despiked_profile.png')  

#### Save despiked data

In [ ]:
data_despiked.to_csv('./output_data/despiked_data.csv')

## Normalize mag data

In [ ]:
data_despiked["normalized_mag"] = normalize_mag(data_despiked.cleaned_mag)

##### Profile of normalized magnetic anaomly data

In [ ]:
y = data_despiked.normalized_mag
name = 'Detrended, Despiked, & Normalized Magnetic Anomlay vs. Distance'
normalized_profile = plot_profile(x, y, direction, name)

st_dev = data_despiked.normalized_mag.std() #standard deviation
data_despiked.normalized_mag[data_despiked.normalized_mag>5*st_dev]=np.nan
print(st_dev)

#### Save normalized profile  

In [ ]:
normalized_profile.savefig('./profiles/normalized_profile.png')

#### Save normalized data

In [ ]:
data_despiked.to_csv('./output_data/normalized_data.csv')

## Perform rolling average using the window mean

In [ ]:
#seconds = '3S' # 3 - quantity of seconds - S denotes seconds
seconds = '3S' # 3 - quantity of seconds - S denotes seconds
time = 3
points = 100 # number of points used in average calculation
#points = 500 # number of points used in average calculation

rolling_average_out = rolling_average(data_despiked, direction, seconds, points)

#### Profile of rolling average

In [ ]:
name = 'Magnetic Anomaly (Rolling Average) reduced data to  '+ str(time)+' second increments using a '+str(points)+' point window vs. Distance'
rolling_average_profile = rolling_profile(rolling_average_out, direction, name)

#### Save rolling average profile 

In [ ]:
rolling_average_profile.savefig('./profiles/rolling_average_'+str(points)+'_profile.png')

#### Save rolling average output data

In [ ]:
rolling_average_out.to_csv('./output_data/rolling_average_data.csv')

## Import Extra Data For Field Map

In [ ]:
file_in_1 = './input_data/TB_all.csv'
data_1 = import_data(file_in_1)

file_in_2 = './input_data/TB_outline_UTM.csv'
data_2 = import_data(file_in_2)

#### Create Location Map

In [ ]:
############################### BEGIN USER INPUTS #############################

zoom_level = 15  #Don't go higher than 18 if avoidable
map_edge_buffer = 1000 # USE 500 - 3000 for big UTM map
map_background = 'satellite' #‘satellite’, ‘terrain’, ‘street’, ‘only_streets'
 
############################### END USER INPUTS ###############################

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> OUTPUT <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#  

location_map = plot_transects_UTM(data_1, data_2, zoom_level, map_edge_buffer, map_background)

location_map.savefig('./maps/magnetic_location_map_UTM.png')

#### Inset map

In [ ]:
def UTM_inset_map(data, data_2, point, zoom, buffer, style):
    fig = plt.figure(figsize=(7,7))
    tiler = GoogleTiles(style=style)
    mercator = tiler.crs
    crs = ccrs.UTM(zone="12N")
    ax = plt.axes(projection=crs)
    
    #Set map extent
    easting = data.easting
    northing = data.northing
    easting_min = min(easting) - buffer
    easting_max = max(easting) + buffer
    northing_min = min(northing) - buffer
    northing_max = max(northing) + buffer
    extent = (easting_min, easting_max, northing_min, northing_max)
    ax.set_extent(extent,crs=crs)

    ax.add_image(tiler, zoom)
        
    plt.plot(easting, northing, marker='.', color='k', linestyle='None',
             markersize=0.2, alpha=0.7, transform=crs)
    
    plt.plot(data_2.easting, data_2.northing, marker='.', color='whitesmoke', 
             linestyle='-', markersize=0.1, alpha=0.7, transform=crs)

    plt.plot(point[0],point[1], marker='s', markersize = 20, color ='red', transform=crs) 
    plt.xlabel('Easting (m)' )
    plt.ylabel('Northing (m)')
    
    return fig

#### Map Inset Inputs

In [ ]:
############################### BEGIN USER INPUTS #############################
line = 3
zoom_level = 15  
map_edge_buffer = 150 
map_background = 'satellite' 
 
############################### END USER INPUTS ###############################

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> OUTPUT <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#  
point = (391000,4871690)
inset_map = UTM_inset_map(data_1, data_2, point, zoom_level, map_edge_buffer, map_background)

inset_map.savefig('./maps/inset_map.png')

#### Import Topography Profile Data

In [ ]:
topo_file = './input_data/TB_line_topo.csv'
topo_data = import_data(topo_file)
topo_data['el'] = topo_data.el_ft /3.2808399 #convert feet to meters
print(topo_data)

#### Reproject Topographic Coordinates From Lat/Lon To UTM

In [ ]:
#REPROJECTION INPUTS
#** always check the proper UTM zone (default = North,12 (Midwest U.S))** - Only CHANGE REPROJECTION INPUTS IF NEEDED
zone = 12
projection = 'utm'
ellipse = 'WGS84'
units = 'm'
definitions = 'no_defs'
hemisphere = 'north'

#TRASNFORMATION DIRECTION
#inverse='True' #for UTM to Lat/Lon
inverse='False' #for Lat/Lon to UTM

#REPROJECTION FUNCTION
myProj = Proj("+proj="+str(projection)+' '+"+zone="+str(zone)+\
              ', '+'+'+str(hemisphere)+' '+"+ellps="+str(ellipse)+\
              ' '+"+datum="+str(ellipse)+' '+"+units="+str(units))

reproject(topo_data, myProj, inverse)
print(topo_data)

#### Magnetic Profile With Topographic Profile And Location Map Insert

In [ ]:
def complete_profile(data, data_2, map_image, zoom, position, box, direction, name):
    fig = plt.figure(figsize=(15, 7))
    ax1 = fig.add_subplot(111)
    min_east = 388000 #actual value = 388200 from line 6aw_and 5b_2
    max_east = 396800 #actual value = 396712 from line 6aw_and 5b_2
    #ax3 = fig.add_subplot()
    if direction == 'ns':
        p1, = ax1.plot(data.northing,data.cleaned_mag, linestyle='-', color='blue',label = 'Magnetic Anomaly (nT)')
        plt.title(str(name))
        ax1.set_ylabel('Magnetic Anomaly (nT)')
        ax1.set_ylim(-700,1400)
        ax1.set_xlabel('Northing (m)')
        ax1.axhline(0,color='k',linestyle='--')
        ax1.yaxis.label.set_color(p2.get_color())
        ax1.yaxis.label.set_fontsize(14)
        ax1.ticklabel_format(useOffset=False, style='plain')
        imagebox = OffsetImage(map_image, zoom = zoom)
        #Annotation box for solar pv logo
        #Container for the imagebox referring to a specific position *xy*.
        ab = AnnotationBbox(imagebox, (position[0], position[1]), frameon = False)
        ax1.add_artist(ab)
        ax1.tick_params(axis='y', colors=p1.get_color(), labelsize=14)

        #add rectangle to plot
        ax1.add_patch(Rectangle((box[0], box[1]), box[2], box[3], ls="--", ec ="red", fill=False))
        
        #Add third axis to plot
        ax2 = ax1.twinx() 
        p2, = ax2.plot(data_2.northing,data_2.el, ls="-", color='black', linewidth=1, label = 'Elevation (masl)')
        ax2.grid(False) # turn off grid #2
        ax2.set_ylabel('Elevation (m)')
        ax2.set_ylim(1450,1800)
        ax2.yaxis.label.set_color(p2.get_color())
        ax2.yaxis.label.set_fontsize(14)
        ax2.tick_params(axis='y', colors=p2.get_color(), labelsize=14)

    elif direction == 'ew':
        p1, = ax1.plot(data.easting,data.cleaned_mag,linestyle='-', color='blue', label = 'Magnetic Anomaly (nT)')
        plt.title(str(name))
        ax1.set_ylabel('Magnetic Anomaly (nT)')
        ax1.set_ylim(-700,1400)
        ax1.set_xlabel('Easting (m)')
        ax1.axhline(0,color='k',linestyle='--')
        ax1.yaxis.label.set_fontsize(14)
        ax1.ticklabel_format(useOffset=False, style='plain')
        imagebox = OffsetImage(map_image, zoom = zoom)
        #Annotation box for solar pv logo
        #Container for the imagebox referring to a specific position *xy*.
        ab = AnnotationBbox(imagebox, (position[0], position[1]), frameon = False)
        ax1.add_artist(ab)
        ax1.tick_params(axis='y', colors=p1.get_color(), labelsize=14)
        
        #add rectangle to plot
        ax1.add_patch(Rectangle((box[0], box[1]), box[2], box[3], ls="--", ec ="red", fill=False))
        
        #Add third axis to plot
        ax2 = ax1.twinx() 
        p2, = ax2.plot(data_2.easting, data_2.el, ls="-", color='black', linewidth=1, label = 'Elevation (masl)')
        ax2.grid(False) # turn off grid #2
        ax2.set_ylabel('Elevation (m)')
        ax2.set_ylim(1450,1800)
        ax2.yaxis.label.set_color(p2.get_color())
        ax2.yaxis.label.set_fontsize(14)
        ax2.tick_params(axis='y', colors=p2.get_color(), labelsize=14)
        
        # ask matplotlib for the plotted objects and their labels
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='upper left')

    return fig

#### Complete Profile Inputs 

In [ ]:
#NEED TO RUN BOTH THE FUNCTION AND INPUT CELLS TO GET IT TO PLOT
image_file = './maps/inset_map.png'
map_image = image.imread(image_file)
zoom = 0.4
inset_position = (397575, 985)
box_position = (390200,-550,1600,1750) #bottom left corner =(1, 1), width = 2, height = 6))
name = 'Magnetic Anomaly (rolling average) and Elevation vs. Distance'
complete_profile = complete_profile(rolling_average_out, topo_data, map_image, zoom, inset_position, box_position, direction, name)

#### Save Complete Profile

In [ ]:
complete_profile.savefig('./profiles/complete_profile.png')